# Kitti Detector Analysis

In [166]:
## evaluating kitti and lyft
#load a kitti file

def loadKittiDetections():

    import pickle
    import pandas as pd 

    
    #filePath = '/home/itiv/Desktop/KITTI_TRACKING_DATA/KITTI_DATASET_ROOT/training/temp_data/result.pkl'
    filePath = '/run/media/joda/Data/joda/BA/repo/result.pkl'

    
    with open(filePath, 'rb') as f:
        data = pickle.load(f)

    pand_data = []
    for (index,frame_det) in enumerate(data):
        #print(frame_det)
        for det_index in range(len(frame_det['name'])):
            detection = {'frame_id':index,'type':frame_det['name'][det_index],'dimensions':frame_det['dimensions'][det_index],'location':frame_det['location'][det_index],'rot':frame_det['rotation_y'][det_index],'score':frame_det['score'][det_index]}
            pand_data.append(detection)


    detections = {'detections':[]}

    for (index,frame_det) in enumerate(data):
        sample_dict = {'token':index,'timestamp':'-','anns':[]}#{'token':detection_dict['sample_token'],'timestamp':'aaa','anns':[]}
        for det_index in range(len(frame_det['name'])):
            ann_dict = {'instance':'detection','translation':frame_det['location'][det_index],'size':frame_det['dimensions'][det_index],'rotation':frame_det['rotation_y'][det_index],'type':frame_det['name'][det_index],'score':frame_det['score'][det_index]}
            sample_dict['anns'].append(ann_dict)
        detections['detections'].append(sample_dict)   

    return detections



def loadKittiGroundTruth():
    
    #TODo!!!!
    
    ## load ground truth
    import pandas as pd
    #scene: index of scene
    #config: small,middle,large
    
    #label_file = '/home/itiv/Desktop/KITTI_TRACKING_DATA/KITTI_DATASET_ROOT/training/scene/label_02/label.txt'
    
    label_file = '/home/itiv/Desktop/KITTI_TRACKING_DATA/KITTI_DATASET_ROOT/training/scene/label_02/label.txt'

    ground_truth = pd.read_csv(label_file, sep=" ", header=None)
    ground_truth.columns = ["FrameID", "ObjectID", "Type", "Truncation","Occlusion","Alpha","BBox-1","BBox-2","BBox-3","BBox-4","Dim-w","Dim-h","Dim-l","Loc-x","Loc-y","Loc-z","Rot"]

    
    dontCareRows = []
    
    for index, row in ground_truth.iterrows():
        if row['Type'] == 'DontCare':
            dontCareRows.append(index)
        
    
    ground_truth.drop(dontCareRows, inplace=True)
    gt_dict = {'samples':[]}

    currentFrameID = None
    sample_dict = None
    for index, row in ground_truth.iterrows():
        if currentFrameID is None:
            currentFrameID = row['FrameID']
            sample_dict = {'token':currentFrameID,'timestamp':currentFrameID*0.1,'anns':[]}   
        elif currentFrameID != row['FrameID']:
            currentFrameID = row['FrameID']
            gt_dict['samples'].append(sample_dict)
            sample_dict = {'token':currentFrameID,'timestamp':currentFrameID*0.1,'anns':[]}
    
        #todo translation und rotation transformation
    
        T_pose = pose_mat[row['FrameID']]

        point = np.array([row['Loc-x'],row['Loc-y'],row['Loc-z'],1])
        point = T_pose.dot(point)
        
        rot = ego_yaw[row['FrameID']]-row['Rot']
        
        
        #traffic_coords.at[row['FrameID'], 'Car '+str(row['ObjectID'])] = point.tolist()[0:3]
    
        ann_dict = {'instance':row['ObjectID'],'translation':point.tolist()[0:3],'size':[row['Dim-h'],row['Dim-w'],row['Dim-l']],'rotation':rot}
        sample_dict['anns'].append(ann_dict)

    
    
    return gt_dict


detections = loadKittiDetections()
groundTruth = loadKittiGroundTruth()
for detection in detections['detections']:
    #print(detection)
    for annotation in detection['anns']:
        if annotation['score'] < 0.4:
            print(annotation['score'])

FileNotFoundError: [Errno 2] File /home/itiv/Desktop/KITTI_TRACKING_DATA/KITTI_DATASET_ROOT/training/scene/label_02/label.txt does not exist: '/home/itiv/Desktop/KITTI_TRACKING_DATA/KITTI_DATASET_ROOT/training/scene/label_02/label.txt'

# Lyft Detector Analysis

In [5]:
## set vars
LYFT_DATASET_ROOT = '/run/media/joda/Data/joda/BA/Lyft-Dataset/v1.01-train/v1.01-train/'
CONFIG_DIR = "./Lyft-Detector/second.pytorch/second/configs/nuscenes/"
MODEL_DIR = './Lyft-Detector/second.pytorch/second/model/'
#MODEL = 'small' #small,middle,large
VERSION = "v1.0-trainval" #v1.0-test for test-dataset

In [6]:
#### load ground truth data
# Load the SDK
%matplotlib inline
from lyft_dataset_sdk.lyftdataset import LyftDataset

# Load the dataset
# Adjust the dataroot parameter below to point to your local dataset path.
# The correct dataset path contains at least the following four folders (or similar): images, lidar, maps, v1.0.1-train
level5data = LyftDataset(data_path=LYFT_DATASET_ROOT, json_path=LYFT_DATASET_ROOT+'/v1.01-train', verbose=True)


9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 19.9 seconds.
Reverse indexing ...
Done reverse indexing in 4.0 seconds.


In [87]:

def loadDetectionDict(scene_num):
    import json
    from pyquaternion import Quaternion

    plot = True
    
    scene = level5data.scene[scene_num]
    scene_token = scene['token']
    
    
    #scene: index of scene
    #config: small,middle,large

    #det_dict_path = '/home/itiv/Desktop/lyft-kaggle-dataset/train/detections_'+scene_token+'.json'
    det_dict_path = '/run/media/joda/Data/joda/BA/repo/detections_'+scene_token+'.json'
    det_dict = json.load(open(det_dict_path))
            
    detections = {'detections':[]}
    
    for frame in det_dict['results']:
        
        #ego_pose = level5data.get('ego_pose',sample_data['ego_pose_token'])
        #print(frame)
        sample = level5data.get('sample', sample_token)
        sample_data = level5data.get('sample_data', sample['data']['LIDAR_TOP'])
        ego_pose = level5data.get('ego_pose',sample_data['ego_pose_token'])
        
        
        detection_dict = det_dict['results'][frame]
        
        
        sample_dict = {'token':frame,'timestamp':'-','anns':[]}#{'token':detection_dict['sample_token'],'timestamp':'aaa','anns':[]}
        for det_index in range(len(det_dict['results'][frame])):
            yaw = Quaternion(detection_dict[det_index]['rotation']).yaw_pitch_roll[0]
            ann_dict = {'instance':'detection','translation':detection_dict[det_index]['translation'],'size':detection_dict[det_index]['size'],'rotation':yaw,'type':detection_dict[det_index]['detection_name'],'score':detection_dict[det_index]['detection_score']}
            if ann_in_boundries(ego_pose,ann_dict,plot) and (detection_dict[det_index]['detection_name'] == 'car' or detection_dict[det_index]['detection_name'] == 'bus' or detection_dict[det_index]['detection_name'] == 'construction_vehicle' or detection_dict[det_index]['detection_name'] == 'motorcycle' or detection_dict[det_index]['detection_name'] == 'truck' or detection_dict[det_index]['detection_name'] == 'trailer'):
                sample_dict['anns'].append(ann_dict)
        detections['detections'].append(sample_dict)
    
    
    return detections






In [88]:
def load_ground_truth_data(scene_num):
    from pyquaternion import Quaternion
    import operator
    import math
    import numpy as np


    scene = level5data.scene[scene_num]
    scene_token = scene['token']

    sample_token = scene['first_sample_token']
    sample = level5data.get('sample', sample_token)
    sample_data = level5data.get('sample_data', sample['data']['LIDAR_TOP'])
    ego_pose = level5data.get('ego_pose',sample_data['ego_pose_token'])


    trans_offset = ego_pose['translation']


    gt_dict = {'ground_truth':[]}
    plot = True

    for index in range(scene['nbr_samples']):
        sample = level5data.get('sample', sample_token)

          

        sample_token = sample['next']
        
        sample_dict = {'token':sample_token,'timestamp':sample['timestamp'],'anns':[]}

        sample_data = level5data.get('sample_data', sample['data']['LIDAR_TOP'])
        ego_pose = level5data.get('ego_pose',sample_data['ego_pose_token'])
        for ann_token in sample['anns']:
            ann = level5data.get('sample_annotation', ann_token)
            
            if ann_in_boundries(ego_pose,ann,plot) and (ann['category_name'] == 'car' or ann['category_name'] == 'bus' or ann['category_name'] == 'construction_vehicle' or ann['category_name'] == 'motorcycle' or ann['category_name'] == 'truck' or ann['category_name'] == 'trailer'):     
                yaw = Quaternion(ann['rotation']).yaw_pitch_roll[0]
                ann_dict = {'instance':ann['instance_token'],'translation':ann['translation'],'size':ann['size'],'rotation':yaw,'type':ann['category_name']}
                sample_dict['anns'].append(ann_dict)

        gt_dict['ground_truth'].append(sample_dict)
        
        
    return gt_dict

det_bound  = [150,100,100,100]
def ann_in_boundries(ego,ann,plot):
    from pyquaternion import Quaternion
    import matplotlib.pyplot as plt
    import numpy as np

    model = 'middle'
    
    '''
    ###TODO!!!!!!!!!!
    #detection boundries TODO filter ground truth out of detection boundary!!!
    det_bound = [50,50,50,50] #dist to front,right,back,left
    if model == 'middle':
        det_bound = [100,70,70,70]
    elif model == 'large':
        det_bound  = [150,100,100,100]
    elif model == 'smallv2':
        det_bound  = [50,10,10,10]
    elif model == 'middlev2':
        det_bound  = [100,10,10,10]
    elif model == 'largev2':
        det_bound  = [150,10,10,10]
        
    '''

    width = det_bound[1]+det_bound[3]
    length = det_bound[0]+det_bound[2]
    
    center_dist = det_bound[0]-length/2 
    
    ego_yaw = Quaternion(ego['rotation']).yaw_pitch_roll[0]
    ego_x_center = ego['translation'][0]
    ego_y_center = ego['translation'][1]
    
    boundary_box_center_x = np.cos(ego_yaw)*center_dist+ego_x_center
    boundary_box_center_y = np.sin(ego_yaw)*center_dist+ego_y_center
    
        

    traffic_yaw = Quaternion(ann['rotation']).yaw_pitch_roll[0]
    traffic_x_center = ann['translation'][0]
    traffic_y_center = ann['translation'][1]
    traffic_width = ann['size'][0]
    traffic_height = ann['size'][1]
    
    ann_rect = []
    ego_rect = []
    
    ann_rect.append([ann['translation'][0],ann['translation'][1],ann['size'][1],ann['size'][0],traffic_yaw])
    ego_rect.append([boundary_box_center_x,boundary_box_center_y,length,width,ego_yaw])
    
    
    ann_rect = np.array(ann_rect)
    ego_rect = np.array(ego_rect)

    ego_pos = rect_polygon(ego_x_center,ego_y_center,traffic_height,traffic_width,ego_yaw)
    
    
    iou = intersection_over_union(ann_rect,ego_rect,plot)
    
    #print(iou)
    
    return (iou[0,0]>0)




def rect_polygon(x, y, width, height, angle):
    from shapely.geometry import Polygon
    from shapely.affinity import rotate, translate
    import math

    """Return a shapely Polygon describing the rectangle with centre at
    (x, y) and the given width and height, rotated by angle quarter-turns.

    """
    w = width / 2
    h = height / 2
    p = Polygon([(-w, -h), (w, -h), (w, h), (-w, h)])
    return translate(rotate(p, math.degrees(angle)), x, y)


def intersection_over_union(rects_a, rects_b,plot=False):
    import rtree.index
    from shapely.affinity import rotate, translate
    from shapely.geometry import Polygon
    import numpy as np
    import matplotlib.pyplot as plt
    import math

    """Calculate the intersection-over-union for every pair of rectangles
    in the two arrays.

    Arguments:
    rects_a: array_like, shape=(M, 5)
    rects_b: array_like, shape=(N, 5)
        Rotated rectangles, represented as (centre x, centre y, width,
        height, rotation in quarter-turns).

    Returns:
    iou: array, shape=(M, N)
        Array whose element i, j is the intersection-over-union
        measure for rects_a[i] and rects_b[j].

    """
    m = len(rects_a)
    n = len(rects_b)
    if m > n:
        # More memory-efficient to compute it the other way round and
        # transpose.
        return intersection_over_union(rects_b, rects_a,plot).T

    # Convert rects_a to shapely Polygon objects.
    polys_a = [rect_polygon(*r) for r in rects_a]

    # Build a spatial index for rects_a.
    index_a = rtree.index.Index()
    for i, a in enumerate(polys_a):
        index_a.insert(i, a.bounds)

        
    # Find candidate intersections using the spatial index.
    iou = np.zeros((m, n))
    for j, rect_b in enumerate(rects_b):
        b = rect_polygon(*rect_b)
        if plot:
            x,y = b.exterior.xy
            if x[0] < 500 and y[0] < 500:
                plt.plot(x,y,'g-')
        for i in range(len(polys_a)):#index_a.intersection(b.bounds):
            a = polys_a[i]
            if(j == 1 and plot):
                x,y = a.exterior.xy
                if x[0] < 500 and y[0] < 500:
                    plt.plot(x,y,'r-')
        
            intersection_area = a.intersection(b).area
            if intersection_area:
                iou[i, j] = intersection_area / a.union(b).area
    
    if plot:
        plt.show()
    return iou




In [52]:
for scene_num in range(100):
    scene = level5data.scene[scene_num]
    scene_token = scene['token']
    if scene_token == "e6dfbcf01ce548fa1550d2453fece66f33fefc5df96611fa4e80800b8ab40eca":
        print(scene_num)
    if scene_token == "e59a765505ae791691b95e8fbf3be86e705ac26ad02ce9ffe44ef4b065c02f8f":
        print(scene_num)    

12
41


In [53]:

sample_token = 'fd0dd65e88dc78ad640139df6d594dc60ae54555c8811287348563656e2d7383'
sample = level5data.get('sample', sample_token)
print(sample)

{'next': 'b817cf770cead870eec001d826e6600ce54fda46395938c1006d7d3c91d22d3f', 'prev': '', 'token': 'fd0dd65e88dc78ad640139df6d594dc60ae54555c8811287348563656e2d7383', 'timestamp': 1557869336401715.2, 'scene_token': 'e6dfbcf01ce548fa1550d2453fece66f33fefc5df96611fa4e80800b8ab40eca', 'data': {'CAM_BACK_RIGHT': '301880d24799bb01308faabc45d21ff81cd6df63aeeb50ecac21368e0f2bd085', 'CAM_FRONT': 'df3bf56afaa0ae0b9f880dfbffefface2f7b6c780901de1cefbfa210abbba7c3', 'CAM_FRONT_LEFT': '21c24e130475500da102d3577f96a87877a1ae79f3b874d790ddc22bfffbb2ba', 'LIDAR_TOP': '5a408fdd34d63050bf2f9a5ce69ffac5ab54502c43f01f15a598b8e05534633a', 'CAM_BACK_LEFT': '12479441dab203807328e98f3d518afa31fc555d78a6968fdcd085d24683044a', 'LIDAR_FRONT_LEFT': 'c03f2cbdb367b9da609004c70941c0591281f9ca16fd678501c792025107430b', 'CAM_FRONT_ZOOMED': 'e8164f47706b1923f98232b00b08fdb3fce8d7933b0ce1014715b912285fad51', 'CAM_FRONT_RIGHT': '3cd887a23d206f2d8a2dac98e5f7aeca781520f1ed66be0dea869c949bcf4a77', 'CAM_BACK': '358a3ed0251ce7

In [136]:
detections = loadDetectionDict(12)
groundTruth = load_ground_truth_data(12)


In [163]:


def eval_detector(scenes,model_identifier,detections,ground_truth):
    import numpy as np
    import json
    
    #scenes = [12,41]

    sceneRanking = []
    treshold_results = []

    for treshold_step in range(11): 

        TRESHOLD = treshold_step*0.1

        print(TRESHOLD)
        Tres_TP = 0
        Tres_FP = 0
        Tres_FN = 0 

        for scene_num in scenes:
            detections = loadDetectionDict(scene_num)
            groundTruth = load_ground_truth_data(scene_num)

            for detData in detections['detections']:
                for gtData in groundTruth['ground_truth']:
                    if detData['token'] == gtData['token']:

                        dets_rect = []
                        for det in detData['anns']:
                            dets_rect.append([det['translation'][0],det['translation'][1],det['size'][1],det['size'][0],det['rotation']])

                        gts_rect = []
                        for gt in gtData['anns']:
                            gts_rect.append([gt['translation'][0],gt['translation'][1],gt['size'][1],gt['size'][0],gt['rotation']])

                        print('analyzing: '+detData['token'])

                        iou = intersection_over_union(dets_rect,gts_rect,True)

                        tp_ranking = []

                        TP = 0
                        FP = 0
                        FN = 0    

                        rescan = True    

                        while rescan and iou.shape[0] > 0:
                            rescan = False
                            for gtIndex in range(iou.shape[1]):
                                column = iou[:,gtIndex]

                                detIndex = np.where(column == np.amax(column))[0][0] 
                                row = iou[detIndex]
                                validationGtIndex = np.where(row == np.amax(row))[0][0]
                                iou_val = max(row)

                                #print('Match at: '+str(gtIndex)+' '+str(detIndex)+" validation: "+str(validationGtIndex))

                                if validationGtIndex == gtIndex and iou_val > TRESHOLD:
                                    rescan = True
                                    TP += 1
                                    tp_ranking.append({'gt':gtIndex,'det':detIndex,'iou':iou_val,'type':'TP'})
                                    iou = np.delete(iou, gtIndex, 1) 
                                    iou = np.delete(iou, detIndex, 0) 
                                    break


                        FP = iou.shape[0]   
                        FN = iou.shape[1] 
                        TP = len(tp_ranking)

                        Tres_TP += TP
                        Tres_FN += FN
                        Tres_FP += FP

                        sceneRanking.append({'IOU_Treshold':TRESHOLD,'TP':TP,'FN':FN,'FP':FP})

                        #print('TP: '+str(TP)+'  FN: '+str(FN)+' FP: '+str(FP))


        treshold_results.append({'IOU_Treshold':TRESHOLD,'TP':Tres_TP,'FN':Tres_FN,'FP':Tres_FP}) 



    print(sceneRanking)
    print(treshold_results)


    #model_identifier = 'test'

    with open(model_identifier+'_results.json', 'w') as fp:
        json.dump(treshold_results, fp)
    with open(model_identifier+'_sceneresults.json', 'w') as fp:
        json.dump(sceneRanking, fp)    


    value = 0
    for result in treshold_results:
        value += result['TP']/(result['TP']+result['FN']+result['FP'])


    mAP = value/11

    print(mAP)

    with open("mAP_results.txt", "a") as myfile:
        myfile.write((model_identifier))
        myfile.write(str(mAP))


In [164]:
eval_detector("test")

0.0
analyzing: b817cf770cead870eec001d826e6600ce54fda46395938c1006d7d3c91d22d3f
analyzing: 9706014f8e0a20109aa54592fc1af90fda97188467f9c4391e5a707ee1fe1c43
analyzing: 4add1b08655580f9a85080fba669ae7a11c3a7fbf271f1c051d1412989b0b94e
analyzing: bf7a1ab82564522e40291d7f6e4d0588011770394887cfa67d71cf06b03fc83a
analyzing: de9fec374846ce07984965c6236fbc9dfa63ca00439fa07712f25882926302c6
analyzing: 420a77cd1b567d4e79ff914d55382d12a25d7a5f519e972daf783ac4c6c5a7c7
analyzing: 605eda9a5620c5c4b7609e2a62e6346fd1337d9a20133371aeb95a209cd41cb8
analyzing: bf11ca875f08ab1078fa0f10776edf29f1774be1ef131f13f84a39cc7e87ad22
analyzing: c09a0c9b14259c1410bed9807aeb2784d928726269c80dd7aef86ce6be7e2485
analyzing: c9dee488b993c016a10d75e18da61004a1fc38976e98dc4e3c9d81360f6820bd
analyzing: b89fdd111eb5aa3b55e7ec2ea8bedfda0fd1698944de4bbcbdea9a230e655462
analyzing: 496f62a3d63d1e231f0e21917423cb38b5238cefca041facaf3a8bb01fb45d92
analyzing: 64de5318ef45a7cd57cfb4113794857038aed1f0b5906b6627795ec78c86f64e
analyzin

analyzing: a2cf5a968650c690eb137c4222a7c4b5f860c1ef6825e83d1b68c7ef17cfe4e2
analyzing: f6197d1c44f5ac645664dcd6756d13237ca9bb1da64d7cb234e3d417ea092240
analyzing: 6cd8a50236f3c6438a71aaa4684ea005b8ac3bdeb99443a885ae98e01da61700
analyzing: 42cf025c34f8311b5ff751217b7ff2265dd1d96ff827db88fb04799ba6f45e0f
analyzing: 3ebce3cc1171fa392566aa11d1c9fea7d97c30d9bc4b91c9fdbc8762058659e2
analyzing: d754b9831b9c46e03fc3281e353306c93301876b6633f3cff1f3a071d0529e44
analyzing: a97b99150d43fd0f00276410c1217f81c7bd6a4b250b38aa1dc9ea496fba8ce9
analyzing: 56a95c88c3d336b64ccf9b8966b77070f0838e0fc0907b9c44e4f603c3f0d00c
analyzing: a6dcd126a52c0869cd853cd2599f7e9ecaa4cfb2b8b16c837463c1d3bc5f2513
analyzing: c731afa6247113c6d9cd358ab968052b21255541e15c2e362ff55170c9fd8407
analyzing: 5261975231cdecf5ee157233796875babd0f66c7d21ac130d639dcefc443199b
analyzing: c5d84ac2099244dbd49142920dd5214614e18117805148d8bf1b62af71a0d844
analyzing: d7662442daf847cd404c6b4d69bab72dce3ee5a81d5f606c1ecc98c4c4ada1dc
analyzing: 6

analyzing: b817cf770cead870eec001d826e6600ce54fda46395938c1006d7d3c91d22d3f
analyzing: 9706014f8e0a20109aa54592fc1af90fda97188467f9c4391e5a707ee1fe1c43
analyzing: 4add1b08655580f9a85080fba669ae7a11c3a7fbf271f1c051d1412989b0b94e
analyzing: bf7a1ab82564522e40291d7f6e4d0588011770394887cfa67d71cf06b03fc83a
analyzing: de9fec374846ce07984965c6236fbc9dfa63ca00439fa07712f25882926302c6
analyzing: 420a77cd1b567d4e79ff914d55382d12a25d7a5f519e972daf783ac4c6c5a7c7
analyzing: 605eda9a5620c5c4b7609e2a62e6346fd1337d9a20133371aeb95a209cd41cb8
analyzing: bf11ca875f08ab1078fa0f10776edf29f1774be1ef131f13f84a39cc7e87ad22
analyzing: c09a0c9b14259c1410bed9807aeb2784d928726269c80dd7aef86ce6be7e2485
analyzing: c9dee488b993c016a10d75e18da61004a1fc38976e98dc4e3c9d81360f6820bd
analyzing: b89fdd111eb5aa3b55e7ec2ea8bedfda0fd1698944de4bbcbdea9a230e655462
analyzing: 496f62a3d63d1e231f0e21917423cb38b5238cefca041facaf3a8bb01fb45d92
analyzing: 64de5318ef45a7cd57cfb4113794857038aed1f0b5906b6627795ec78c86f64e
analyzing: 3

analyzing: f6197d1c44f5ac645664dcd6756d13237ca9bb1da64d7cb234e3d417ea092240
analyzing: 6cd8a50236f3c6438a71aaa4684ea005b8ac3bdeb99443a885ae98e01da61700
analyzing: 42cf025c34f8311b5ff751217b7ff2265dd1d96ff827db88fb04799ba6f45e0f
analyzing: 3ebce3cc1171fa392566aa11d1c9fea7d97c30d9bc4b91c9fdbc8762058659e2
analyzing: d754b9831b9c46e03fc3281e353306c93301876b6633f3cff1f3a071d0529e44
analyzing: a97b99150d43fd0f00276410c1217f81c7bd6a4b250b38aa1dc9ea496fba8ce9
analyzing: 56a95c88c3d336b64ccf9b8966b77070f0838e0fc0907b9c44e4f603c3f0d00c
analyzing: a6dcd126a52c0869cd853cd2599f7e9ecaa4cfb2b8b16c837463c1d3bc5f2513
analyzing: c731afa6247113c6d9cd358ab968052b21255541e15c2e362ff55170c9fd8407
analyzing: 5261975231cdecf5ee157233796875babd0f66c7d21ac130d639dcefc443199b
analyzing: c5d84ac2099244dbd49142920dd5214614e18117805148d8bf1b62af71a0d844
analyzing: d7662442daf847cd404c6b4d69bab72dce3ee5a81d5f606c1ecc98c4c4ada1dc
analyzing: 60fc8aedc038493c542576503505885d8565aecff3c195fc1eb6c35702928efa
analyzing: 1

analyzing: b817cf770cead870eec001d826e6600ce54fda46395938c1006d7d3c91d22d3f
analyzing: 9706014f8e0a20109aa54592fc1af90fda97188467f9c4391e5a707ee1fe1c43
analyzing: 4add1b08655580f9a85080fba669ae7a11c3a7fbf271f1c051d1412989b0b94e
analyzing: bf7a1ab82564522e40291d7f6e4d0588011770394887cfa67d71cf06b03fc83a
analyzing: de9fec374846ce07984965c6236fbc9dfa63ca00439fa07712f25882926302c6
analyzing: 420a77cd1b567d4e79ff914d55382d12a25d7a5f519e972daf783ac4c6c5a7c7
analyzing: 605eda9a5620c5c4b7609e2a62e6346fd1337d9a20133371aeb95a209cd41cb8
analyzing: bf11ca875f08ab1078fa0f10776edf29f1774be1ef131f13f84a39cc7e87ad22
analyzing: c09a0c9b14259c1410bed9807aeb2784d928726269c80dd7aef86ce6be7e2485
analyzing: c9dee488b993c016a10d75e18da61004a1fc38976e98dc4e3c9d81360f6820bd
analyzing: b89fdd111eb5aa3b55e7ec2ea8bedfda0fd1698944de4bbcbdea9a230e655462
analyzing: 496f62a3d63d1e231f0e21917423cb38b5238cefca041facaf3a8bb01fb45d92
analyzing: 64de5318ef45a7cd57cfb4113794857038aed1f0b5906b6627795ec78c86f64e
analyzing: 3

...

analyzing: f6197d1c44f5ac645664dcd6756d13237ca9bb1da64d7cb234e3d417ea092240
analyzing: 6cd8a50236f3c6438a71aaa4684ea005b8ac3bdeb99443a885ae98e01da61700
analyzing: 42cf025c34f8311b5ff751217b7ff2265dd1d96ff827db88fb04799ba6f45e0f
analyzing: 3ebce3cc1171fa392566aa11d1c9fea7d97c30d9bc4b91c9fdbc8762058659e2
analyzing: d754b9831b9c46e03fc3281e353306c93301876b6633f3cff1f3a071d0529e44
analyzing: a97b99150d43fd0f00276410c1217f81c7bd6a4b250b38aa1dc9ea496fba8ce9
analyzing: 56a95c88c3d336b64ccf9b8966b77070f0838e0fc0907b9c44e4f603c3f0d00c
analyzing: a6dcd126a52c0869cd853cd2599f7e9ecaa4cfb2b8b16c837463c1d3bc5f2513
analyzing: c731afa6247113c6d9cd358ab968052b21255541e15c2e362ff55170c9fd8407
analyzing: 5261975231cdecf5ee157233796875babd0f66c7d21ac130d639dcefc443199b
analyzing: c5d84ac2099244dbd49142920dd5214614e18117805148d8bf1b62af71a0d844
analyzing: d7662442daf847cd404c6b4d69bab72dce3ee5a81d5f606c1ecc98c4c4ada1dc
analyzing: 60fc8aedc038493c542576503505885d8565aecff3c195fc1eb6c35702928efa
analyzing: 1

analyzing: 93098ae2bec49f651164101abed90efbc9a31c154ff811220029ce935bb3c6ba
analyzing: e42741d68903dfae47223e0996e48d1e8ce104b05526d093a608895c6dbe9e90
analyzing: 702c56118028d21064072c8f03059b0c7c096971c20c4986cbef374257cc1c3b
analyzing: 508df9ce59e960e70697096c58fb6ea6d9a1f009fbde2ec9336e284b90d796b3
analyzing: a458924d8f77f2eaf3198f444cef857d17a65df1fdb5c1db4f37fce3e9c7b52f
analyzing: 3de670847535abfac6608e001d03df83287b6e4097dacbb4884d98a21318bd4a
analyzing: 703089bfabc68387a464cd9450a538e84822569a47aa624e4698e7f41dfa655b
analyzing: 89e929849b61822dcf8aa94b0cd6770fefd9385d4efb94e6acdb51e4be435f8b
analyzing: 8500787efa588597da810da1385a89aac0e460a131701f2e5e023d83b4188497
analyzing: 5baa59369191d503b65b274294986d979dcc53a204f64b24d74a257315610333
analyzing: 83167e3fb54c3d13dade9740554a876031bf60e0acb46527f8cb2615a3902b13
analyzing: 126ccd7ca25e6c52f1e41cbd8505194cdeb36fa7d4c3a6d62594c8d094cabc42
analyzing: 81038bf08644b71d24f33dca0748ba743147462d34585f75c611eb4e0b296fac
analyzing: 4

analyzing: bdf674b3f4008be79548602382c7ec5a5939b4420cefc94a883c82f4b7555582
analyzing: 44a9dc7b3ff05cec1de1fedde5f9a749a2479bb433238683cb44735f6505ed24
analyzing: 894e819be5209d033cfb16474612135b2220ad72b0e470804f6b8a7baaaba973
analyzing: 4fd4b24ce03a28af65548b2aae88ce594d6437ab407b822cf460e11035a2c2d1
analyzing: 5ff376b95bd779894ac879be52f8542e10102de05034de99416e7d3ad99bf16d
analyzing: b9b609dc9e67440eb17e9a3609f7bca5ec19bc2af33323f7f98896150a951bc5
analyzing: d9ed93b69a2de084d8688c10a42489fa387f26b3ed3fff65768b5af0ee27e9df
analyzing: 11210eb311f85c4ee2285d13a7c330c6668b938f829180b6e5d465a36bc66714
analyzing: e642347f41f374154618c51bc548287dccc37818f7750e6c66d09da06f71bb98
analyzing: 3a107523a438bf7acf0a5bbb47cee727a8b4870077c4c6354eb1b5fb398ac5dc
analyzing: b96d0b9dc78e247bc667550266064066aed70e91beaccc832e845b17b80cfa34
analyzing: f0d7a620f295f4b43ca2d659b24f06327bc982f535f54af7c5fa2d3e54239f3f
analyzing: 746416e96e54484cbb77f50bdc82beda86c34518469fb6913fafc5969e6b7aa7
analyzing: 5

analyzing: b817cf770cead870eec001d826e6600ce54fda46395938c1006d7d3c91d22d3f
analyzing: 9706014f8e0a20109aa54592fc1af90fda97188467f9c4391e5a707ee1fe1c43
analyzing: 4add1b08655580f9a85080fba669ae7a11c3a7fbf271f1c051d1412989b0b94e
analyzing: bf7a1ab82564522e40291d7f6e4d0588011770394887cfa67d71cf06b03fc83a
analyzing: de9fec374846ce07984965c6236fbc9dfa63ca00439fa07712f25882926302c6
analyzing: 420a77cd1b567d4e79ff914d55382d12a25d7a5f519e972daf783ac4c6c5a7c7
analyzing: 605eda9a5620c5c4b7609e2a62e6346fd1337d9a20133371aeb95a209cd41cb8
analyzing: bf11ca875f08ab1078fa0f10776edf29f1774be1ef131f13f84a39cc7e87ad22
analyzing: c09a0c9b14259c1410bed9807aeb2784d928726269c80dd7aef86ce6be7e2485
analyzing: c9dee488b993c016a10d75e18da61004a1fc38976e98dc4e3c9d81360f6820bd
analyzing: b89fdd111eb5aa3b55e7ec2ea8bedfda0fd1698944de4bbcbdea9a230e655462
analyzing: 496f62a3d63d1e231f0e21917423cb38b5238cefca041facaf3a8bb01fb45d92
analyzing: 64de5318ef45a7cd57cfb4113794857038aed1f0b5906b6627795ec78c86f64e
analyzing: 3

analyzing: f6197d1c44f5ac645664dcd6756d13237ca9bb1da64d7cb234e3d417ea092240
analyzing: 6cd8a50236f3c6438a71aaa4684ea005b8ac3bdeb99443a885ae98e01da61700
analyzing: 42cf025c34f8311b5ff751217b7ff2265dd1d96ff827db88fb04799ba6f45e0f
analyzing: 3ebce3cc1171fa392566aa11d1c9fea7d97c30d9bc4b91c9fdbc8762058659e2
analyzing: d754b9831b9c46e03fc3281e353306c93301876b6633f3cff1f3a071d0529e44
analyzing: a97b99150d43fd0f00276410c1217f81c7bd6a4b250b38aa1dc9ea496fba8ce9
analyzing: 56a95c88c3d336b64ccf9b8966b77070f0838e0fc0907b9c44e4f603c3f0d00c
analyzing: a6dcd126a52c0869cd853cd2599f7e9ecaa4cfb2b8b16c837463c1d3bc5f2513
analyzing: c731afa6247113c6d9cd358ab968052b21255541e15c2e362ff55170c9fd8407
analyzing: 5261975231cdecf5ee157233796875babd0f66c7d21ac130d639dcefc443199b
analyzing: c5d84ac2099244dbd49142920dd5214614e18117805148d8bf1b62af71a0d844
analyzing: d7662442daf847cd404c6b4d69bab72dce3ee5a81d5f606c1ecc98c4c4ada1dc
analyzing: 60fc8aedc038493c542576503505885d8565aecff3c195fc1eb6c35702928efa
analyzing: 1